This notebook is used to determine which strategy is the best for any particular asset.

In [14]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:
import pandas as pd

df = pd.read_csv("../data/processed/starting_portfolio_2months.csv")

Below, we use a longer time period just for the calculation of technical indicators s.t. we have values from our desired day 1 of backtesting. We will only run backtesting on a shorter time frame.

In [16]:
import yfinance as yf

start_date = "2023-03-01"  # Longer time period for calculation of technical indicators
true_start_date = "2025-01-01"  # True first day of backtesting
end_date = "2025-03-01"  # Last day is exclusive

portfolio = {}
for asset, weight in list(map(list, df.values)):
    portfolio[asset] = {
        "data": yf.Ticker(asset).history(start=start_date, end=end_date, actions=False),
        "weight": weight,
    }

In [17]:
# Commission fee based on Webull, which is known for low commission fees:
# https://www.webull.com.sg/pricing
# Regular and Extended Hours (04:00 - 20:00 EST)
# 0.025%*Total Trade Amount (Min. USD 0.50)

def commission(order_size, price):
    return max(0.5, abs(order_size) * price * 0.00025)

We calculate all the technical indicators required for all the strategies first before truncating the data. This is so that we have data on e.g. 50-day moving averages even on our desired day 1, but we only run backtesting from day 1 onwards.

In [18]:
import pandas_ta as ta


def calculate_technical_indicators(df):
    """
    We calculate all the technical indicators required for all the strategies first
    before truncating the data. This is so that we have data on e.g. 50-day moving
    averages even on our desired day 1, but we only run backtesting from day 1 onwards.
    """

    # Core indicators
    df["EMA_50"] = ta.ema(df["Close"], length=50)
    df["EMA_200"] = ta.ema(df["Close"], length=200)
    df["RSI"] = ta.rsi(df["Close"], length=14)
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=7)

    # Bollinger Bands of length 20
    bbands = ta.bbands(df["Close"], length=20)
    bbands = bbands.rename(
        columns={
            "BBU_20_2.0": "Upper_Band",
            "BBM_20_2.0": "Middle_Band",
            "BBL_20_2.0": "Lower_Band",
            "BBB_20_2.0": "Band_Width",
            "BBP_20_2.0": "Percent_B",
        }
    )

    # Bollinger Bands of length 200
    bbands200 = ta.bbands(df["Close"], length=200)
    bbands200 = bbands200.rename(
        columns={
            "BBU_200_2.0": "Upper_Band_200",
            "BBM_200_2.0": "Middle_Band_200",
            "BBL_200_2.0": "Lower_Band_200",
            "BBB_200_2.0": "Band_Width_200",
            "BBP_200_2.0": "Percent_B_200",
        }
    )

    # MACD
    macd = ta.macd(df["Close"])
    macd = macd.rename(
        columns={
            "MACD_12_26_9": "MACD",
            "MACDh_12_26_9": "Histogram",
            "MACDs_12_26_9": "Signal",
        }
    )

    # Miscellaneous
    df["SMA_20"] = ta.sma(df["Close"], length=20)
    df["STD_20"] = ta.stdev(df["Close"], length=20)
    df["SMA_Volume_10"] = ta.sma(df["Volume"], length=10)
    df["Momentum"] = ta.mom(df["Close"], length=10)

    df = df.join([bbands, bbands200, macd])

    return df

In [19]:
from backtesting import Backtest


def run_backtest(asset, strategy, plot=False):
    data = portfolio[asset]["data"]
    data = calculate_technical_indicators(portfolio[asset]["data"])
    data.index = data.index.values.astype("datetime64[D]")
    data = data.loc[data.index >= true_start_date]
    cash = 1000000 * portfolio[asset]["weight"]
    if asset.endswith("-USD"):  # Crypto
        # https://www.webullpay.com/
        # No Direct Fees: Webull Pay does not charge separate fees for trading cryptocurrencies.
        # Instead, a 1% spread (100 basis points) is included in the buying and selling prices
        # of the crypto assets. This means the cost is integrated into the price you pay or
        # receive when trading.
        bt = Backtest(data, strategy, cash=cash, spread=0.01, finalize_trades=True)
    else:
        bt = Backtest(
            data, strategy, cash=cash, commission=commission, finalize_trades=True
        )
    stats = bt.run()
    if plot:
        bt.plot()
    return bt, stats

## All assets and all strategies

In [31]:
from strategies.backtest.larry_williams_price_action import LarryWilliamsPriceAction
from strategies.backtest.macd_bollinger_bands_mean_reversion import (
    MACDBollingerBandsMeanReversion,
)
from strategies.backtest.mean_reversion import MeanReversion
from strategies.backtest.michael_harris_price_action import MichaelHarrisPriceAction
from strategies.backtest.momentum import Momentum
from strategies.backtest.rsi_divergence import RSIDivergence
from strategies.backtest.scalping import Scalping
from strategies.backtest.volume_spike_reversal import VolumeSpikeReversal
from strategies.backtest.bollinger_bands_breakout import BollingerBandsBreakout

strategies = [
    BollingerBandsBreakout,
    LarryWilliamsPriceAction,
    MACDBollingerBandsMeanReversion,
    MeanReversion,
    MichaelHarrisPriceAction,
    Momentum,
    RSIDivergence,
    Scalping,
    VolumeSpikeReversal,
]

cols = []
all_stats = {}
for asset in portfolio:
    for strategy in strategies:
        bt, stats = run_backtest(asset, strategy, plot=False)
        col = stats.drop(["_equity_curve", "_trades"])
        col["Asset"] = asset
        col["Weight"] = portfolio[asset]["weight"]
        cols.append(col)
        
        all_stats[(asset, strategy)] = (stats, bt)

# NOTE: Many instances of broker cancelling the relative-sized order due to insufficient margin.

Backtest.run:   0%|          | 0/38 [00:00<?, ?bar/s]c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=22: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
Backtest.run:   0%|          | 0/38 [00:00<?, ?bar/s]c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=9: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
Backtest.run:   0%|          | 0/38 [00:00<?, ?bar/s]c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=26: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\JW\AppData\Local\pypoetry\Cache\virtualenvs\cryptoto-IqJLIiLB-py3.12\Lib\site-packages\backtesting\backtesting.py:954: UserWa

In [21]:
results = pd.concat(cols, axis=1).transpose()
results = results.rename(columns={"_strategy":"Strategy"})

In [9]:
results.to_csv("../data/experiments/asset_strategies_2_months_with_tpsl.csv", index=False)

## Example filtering on results

In [10]:
results.columns

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'CAGR [%]',
       'Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio', 'Alpha [%]', 'Beta',
       'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
       'Avg. Drawdown Duration', '# Trades', 'Win Rate [%]', 'Best Trade [%]',
       'Worst Trade [%]', 'Avg. Trade [%]', 'Max. Trade Duration',
       'Avg. Trade Duration', 'Profit Factor', 'Expectancy [%]', 'SQN',
       'Kelly Criterion', 'Strategy', 'Asset', 'Weight', 'Commissions [$]'],
      dtype='object')

In [11]:
idx = results.groupby("Asset")["Return [%]"].idxmax()
results.loc[idx]

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,Strategy,Asset,Weight,Commissions [$]
236,2025-01-01 00:00:00,2025-02-28 00:00:00,58 days 00:00:00,0.0,16801.696482,16801.696482,0.0,-21.995695,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,MACDBollingerBandsMeanReversion,ADA-USD,0.016802,NaN
205,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,28.205128,32292.306852,32292.306852,12.266128,8.852609,111.195269,35.021495,...,15 days 00:00:00,15 days 00:00:00,NaN,12.424242,NaN,NaN,Scalping,ADI,0.028764,15.145803
6,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,46.153846,62674.361177,62674.361177,6.627583,-9.778786,51.384104,50.735616,...,16 days 00:00:00,12 days 00:00:00,2.871404,3.756051,0.467221,0.318439,RSIDivergence,AES,0.058779,55.966471
12,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,10.25641,57202.260297,57202.260297,1.447133,-15.447493,9.728278,2.251944,...,1 days 00:00:00,1 days 00:00:00,NaN,0.775478,6.095489,NaN,MeanReversion,AKAM,0.056386,56.444242
67,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,5.128205,47161.07273,47161.07273,10.208296,-2.105924,87.39985,33.714591,...,1 days 00:00:00,1 days 00:00:00,NaN,10.315284,NaN,NaN,MichaelHarrisPriceAction,BLDR,0.042793,22.380595
171,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,0.0,29253.439182,29253.439182,0.0,-5.166104,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,BollingerBandsBreakout,CARR,0.029253,NaN
79,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,25.641026,46835.903543,46835.903543,13.635765,9.268675,128.408666,35.061724,...,4 days 00:00:00,4 days 00:00:00,NaN,6.761387,1.876931,NaN,Scalping,CBRE,0.041216,44.527055
99,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,0.0,33277.14643,33277.14643,0.0,7.652122,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,BollingerBandsBreakout,CSGP,0.033277,NaN
55,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,15.384615,46395.454325,46411.203739,6.126716,-31.866625,46.847809,10.843857,...,8 days 00:00:00,8 days 00:00:00,NaN,6.221144,NaN,NaN,LarryWilliamsPriceAction,DECK,0.043717,22.381825
197,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,25.641026,30316.799504,31322.46304,4.803808,12.573252,35.415127,25.836332,...,10 days 00:00:00,7 days 00:00:00,3.133106,2.559731,0.489316,0.328551,VolumeSpikeReversal,DXCM,0.028927,30.156951


## Comparison of experiments

In [12]:
import os

path = "../data/experiments/"

results = []
files = os.listdir(path)
for f in files:
    res = pd.read_csv(path + f)
    idx = res.groupby("Asset")["Return [%]"].idxmax()
    best = res.loc[idx]
    total = sum(best["Return [%]"] * best["Weight"])
    results.append([f, total])

pd.DataFrame(results, columns=["Experiment", "Return [%]"])

,Experiment,Return [%]
0,asset_strategies_1_year_no_tpsl.csv,21.106399
1,asset_strategies_1_year_with_tpsl.csv,21.264665
2,asset_strategies_2_months_no_tpsl.csv,4.390247
3,asset_strategies_2_months_with_tpsl.csv,5.088805
